In [1]:
import requests

API_URL = "http://api.netspeak.org/netspeak3/search?query=%s"

class NetSpeak:
    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'}
        self.page = None

    def __getPageContent(self, url):
        return requests.get(url, headers=self.headers).text
        # return self.opener.open(url).read()

    def __rolling(self, url, maxfreq=None):
        if maxfreq:
            webdata = self.__getPageContent(url + "&maxfreq=%s" % maxfreq)
        else:
            webdata = self.__getPageContent(url)
        if webdata:
            # webdata = webdata.decode('utf-8')
            results = [data.split('\t') for data in webdata.splitlines()]
            results = [(data[2], float(data[1])) for data in results]
            lastFreq = int(results[-1][1])
            if lastFreq != maxfreq:
                return results + self.__rolling(url, lastFreq)
            else:
                return []
        else:
            return []

    def search(self, query):
        queries = query.split()
        new_query = []
        for token in queries:
            if token.count('|') > 0:
                new_query.append('[+{0}+]'.format('+'.join(token.split('|'))))
            elif token == '*':
                new_query.append('?')
            else:
                new_query.append(token)
        new_query = '+'.join(new_query)
        url = API_URL % (new_query.replace(' ', '+'))
        return self.__rolling(url)

In [2]:
trigram_dictionary = {}

- NetSpeak 有找到 trigram 時會丟一個 list of a tuple 回來

In [3]:
SE = NetSpeak()
res = SE.search('when the break')
res

[('when the break', 6125.0)]

- NetSpeak 沒找到，丟空 list

In [4]:
res = SE.search('when the ek')
res

[]

## TriGram Error Dectection

In [5]:
SE = NetSpeak()
test = 'when the brack was finished'

def error_trigram(sentence):
    sentence = sentence.split()
    error_candidates = {}
    for i in range(len(sentence) - 2):
        tri_gram = ' '.join(sentence[i:i + 3])
        if tri_gram in trigram_dictionary:
            error_candidates[(tri_gram, i)] = trigram_dictionary[tri_gram]
        else:
            res = SE.search(tri_gram)
            if res:
                trigram_dictionary[tri_gram] = res[0][1]
                error_candidates[(tri_gram, i)] = res[0][1]
            else:
                trigram_dictionary[tri_gram] = 0
                error_candidates[(tri_gram, i)] = 0
    return min(error_candidates.items(), key=lambda x: x[1])[0]

## Find The Best TriGram after Correction 

In [7]:
import Spelling_check_nosiy_channel as cor
from pprint import pprint

## corrected trigram list

In [8]:
def get_correct_list(trigram_tuple):
    trigram = trigram_tuple[0].split(' ') # tri_gram
    idx_trigram = trigram_tuple[1] # i
    cor_tri_list = []
    for idx, word in enumerate(trigram):
        for cor_tuple in cor.correction(word):
            new_trigram = list(trigram)
            cor_word = cor_tuple[0]
            new_trigram[idx] = cor_word
            cor_tri_list.append(' '.join(new_trigram))
    return idx_trigram, cor_tri_list 

## choose the highest trigram form correted trigram list

In [9]:
def score_trigram(trigram_set):
    idx_trigram = trigram_set[0]
    trigram_set = trigram_set[1]
    print(trigram_set)
    scores = {}
    for tri in trigram_set:
        if tri in trigram_dictionary:
            scores[tri] = trigram_dictionary[tri]
        else:
            res = SE.search(tri)
            if res == []:
                trigram_dictionary[tri] = 0
                scores[tri] = 0
            else:
                trigram_dictionary[tri] = res[0][1]
                scores[tri] = res[0][1]
    return idx_trigram, max(scores.items(), key=lambda x: x[1])[0]

                

In [10]:
hit = 0

In [11]:
def answer(score_trigram, sentence):
    s_arr = sentence.split(' ')
    tri_idx = score_trigram[0]
    tri_arr = score_trigram[1].split(' ')
    wrong = ''
    correct = ''
    for idx, word in enumerate(tri_arr):
        if word != s_arr[(tri_idx+idx)]:
            correct = word
            wrong = s_arr[(tri_idx+idx)]
    s_arr[tri_idx:tri_idx+3] = tri_arr
    candidate = []
    for c in cor.correction(wrong):
        candidate.append(c[0])
    print("Error:",wrong)
    print("Candidates:",candidate)
    print("Correction:",correct)
    cor_sentence = ' '.join(s_arr)
    print(sentence,"->",cor_sentence)
    return cor_sentence.strip()

In [12]:
hit = 0

fp = open('lab4.test.1.txt', "r")
lines = fp.readlines()
fp.close()

wrong = []
correct = []

for i in range(len(lines)):
    source = lines[i].split('\t') # wrong \t correct
    wrong.append(source[0].lower())
    correct.append(source[1].strip('\n').lower())
    
for index, sentence in enumerate(wrong):
    best_trigram = score_trigram(get_correct_list(error_trigram(sentence)))
    print(index,"---------------------------------------")
    if answer(best_trigram, sentence) == correct[index]:
        hit = hit + 1
    print("hits :",hit)
    print("\n")

['felt very strang', 'field very strang', 'fell very strang', 'felt very strang', 'felt every strang', 'felt vary strang', 'felt very strange', 'felt very strong', 'felt very staring']
0 ---------------------------------------
Error: strang
Candidates: ['strange', 'strong', 'staring']
Correction: strange
i felt very strang  -> i felt very strange 
hits : 1


['and brake time', 'at brake time', 'it brake time', 'at break time', 'at broken time', 'at broke time', 'at brake time', 'at brake times', 'at brake them']
1 ---------------------------------------
Error: brake
Candidates: ['break', 'broken', 'broke']
Correction: break
at brake time  -> at break time 
hits : 2


['when the brack', 'went the brack', 'whom the brack', 'when the brack', 'when they brack', 'when then brack', 'when the black', 'when the back', 'when the branch']
2 ---------------------------------------
Error: brack
Candidates: ['black', 'back', 'branch']
Correction: black
when the brack was finished  -> when the black

KeyboardInterrupt: 